<a href="https://colab.research.google.com/github/DHIVYASRI-D/Comparing-Transformer-Models-for-Token-Based-Code-Completion-in-Python/blob/main/UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Gradio

In [ ]:
!pip install gradio --quiet


In [ ]:
!pip install transformers gradio sentencepiece


#**final output**


In [ ]:
!pip install -q transformers gradio

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, T5ForConditionalGeneration

# Set device to CPU
device = torch.device("cpu")

# Load GPT-Neo 125M
neo_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/token-completion-models/gptneo-125M", local_files_only=True)
neo_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/token-completion-models/gptneo-125M", local_files_only=True).to(device).eval()

# Load CodeT5-Base
t5_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine-tuned-models/codeT5-base-finetuned", local_files_only=True)
t5_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/fine-tuned-models/codeT5-base-finetuned", local_files_only=True).to(device).eval()

# Load CodeGPT-Small-Py
codegpt_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/token-completion-models/codegpt-small-py", local_files_only=True)
codegpt_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/token-completion-models/codegpt-small-py", local_files_only=True).to(device).eval()

# Load CodeGen-350M-Multi
codegen_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine-tuned-models/codegen-350M-multi", local_files_only=True)
codegen_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/fine-tuned-models/codegen-350M-multi", local_files_only=True).to(device).eval()


In [ ]:
def predict_all_models(prompt, max_tokens):
    neo_out = predict_gptneo(prompt, max_tokens)
    t5_out = predict_codet5(prompt, max_tokens)
    gpt_out = predict_codegpt(prompt, max_tokens)
    codegen_out = predict_codegen(prompt, max_tokens)
    return neo_out, t5_out, gpt_out, codegen_out

def predict_gptneo(prompt, max_tokens):
    inputs = neo_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cpu")
    outputs = neo_model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False, pad_token_id=neo_tokenizer.eos_token_id)
    decoded = neo_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip() if decoded.startswith(prompt) else decoded.strip()

def predict_codet5(prompt, max_tokens):
    input_ids = t5_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).input_ids.to("cpu")
    outputs = t5_model.generate(input_ids, max_new_tokens=max_tokens, do_sample=False, num_beams=1)
    return t5_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

def predict_codegpt(prompt, max_tokens):
    inputs = codegpt_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cpu")
    outputs = codegpt_model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False, pad_token_id=codegpt_tokenizer.eos_token_id)
    decoded = codegpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip() if decoded.startswith(prompt) else decoded.strip()

def predict_codegen(prompt, max_tokens):
    inputs = codegen_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cpu")
    outputs = codegen_model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False, pad_token_id=codegen_tokenizer.eos_token_id)
    decoded = codegen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip() if decoded.startswith(prompt) else decoded.strip()

# UI
with gr.Blocks() as demo:
    gr.Markdown("### 🔮 Code Completion with Multiple Models")

    with gr.Row():
        prompt = gr.Textbox(label="Input Prompt", placeholder="Type your Python code...")
        max_tokens = gr.Slider(1, 10, value=1, step=1, label="Tokens to Predict")

    with gr.Row():
        btn = gr.Button("Generate")

    with gr.Row():
        out_neo = gr.Textbox(label="GPT-Neo 125M")
        out_t5 = gr.Textbox(label="CodeT5-Base")
        out_gpt = gr.Textbox(label="CodeGPT-Small-Py")
        out_codegen = gr.Textbox(label="CodeGen-350M-Multi")

    btn.click(fn=predict_all_models, inputs=[prompt, max_tokens], outputs=[out_neo, out_t5, out_gpt, out_codegen])

demo.launch()

